In [1]:
from fastai.text.all import *

In [2]:
path = untar_data(URLs.IMDB)
path.ls()

(#7) [Path('/storage/data/imdb/README'),Path('/storage/data/imdb/imdb.vocab'),Path('/storage/data/imdb/tmp_lm'),Path('/storage/data/imdb/tmp_clas'),Path('/storage/data/imdb/unsup'),Path('/storage/data/imdb/test'),Path('/storage/data/imdb/train')]

In [3]:
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1)

In [4]:
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxmaj one of the better made for xxup tv biopics , i just wish it had told us more . i have read many biographies and seen other things about the xxmaj carpenters , and this movie did what it could , based on the constraints placed on it by the family . xxmaj xxunk xxmaj gibb did a wonderful job trying to bring xxmaj karen to life . xxmaj one","xxmaj one of the better made for xxup tv biopics , i just wish it had told us more . i have read many biographies and seen other things about the xxmaj carpenters , and this movie did what it could , based on the constraints placed on it by the family . xxmaj xxunk xxmaj gibb did a wonderful job trying to bring xxmaj karen to life . xxmaj one of"
1,"xxmaj this movie is very difficult to understand , why do the couple want to divorce ? xxmaj no reason is given , we do n't know anything about their life in xxmaj lisbon , and even nothing about xxmaj marie 's job . xxmaj we may only understand that a certain bore appeared in this life . xxmaj we do n't even know who took the initiative of asking the divorce","this movie is very difficult to understand , why do the couple want to divorce ? xxmaj no reason is given , we do n't know anything about their life in xxmaj lisbon , and even nothing about xxmaj marie 's job . xxmaj we may only understand that a certain bore appeared in this life . xxmaj we do n't even know who took the initiative of asking the divorce ."
2,"the director xxmaj xxunk is up there with the best camp directors , and this is his best movie . \n\n xxmaj in xxmaj yon xxunk , everything seems to come together . xxmaj there 's nothing good about it , but still manages to be a coherent whole . xxmaj not once does the movie slow down - the action flows on and punches keep on coming . \n\n xxmaj the","director xxmaj xxunk is up there with the best camp directors , and this is his best movie . \n\n xxmaj in xxmaj yon xxunk , everything seems to come together . xxmaj there 's nothing good about it , but still manages to be a coherent whole . xxmaj not once does the movie slow down - the action flows on and punches keep on coming . \n\n xxmaj the weirdest"
3,"slasher movie like the classics ( friday the 13th , xxmaj halloween , xxmaj nightmare on xxmaj elm xxmaj street ) , but it 's surprising and incredibly smart when you think about how the movie got you to that point from the beginning . xxbos xxmaj this tender beautifully crafted production delved deep down bitter sweet into my being . xxmaj the irreverent pupils , the life embittered bus driver and","movie like the classics ( friday the 13th , xxmaj halloween , xxmaj nightmare on xxmaj elm xxmaj street ) , but it 's surprising and incredibly smart when you think about how the movie got you to that point from the beginning . xxbos xxmaj this tender beautifully crafted production delved deep down bitter sweet into my being . xxmaj the irreverent pupils , the life embittered bus driver and the"
4,"o'connor all playing frustrated doctors engaged in a game of musical beds . * from xxrep 4 * xxbos xxmaj despite the fact that there were aspects of this film that i felt were not developed enough , i enjoyed it and would recommend it to others . xxmaj richard xxmaj gere and xxmaj diane xxmaj lane are great in their lead roles . xxmaj the basic premise of the film is","all playing frustrated doctors engaged in a game of musical beds . * from xxrep 4 * xxbos xxmaj despite the fact that there were aspects of this film that i felt were not developed enough , i enjoyed it and would recommend it to others . xxmaj richard xxmaj gere and xxmaj diane xxmaj lane are great in their lead roles . xxmaj the basic premise of the film is that"


In [5]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()],
                               path=path, wd=0.1, model_dir='/storage/models').to_fp16()

In [6]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.531472,4.124944,0.285971,61.864357,38:48


In [7]:
learn.save('1epoch')

Path('/storage/models/1epoch.pth')

In [8]:
learn = learn.load('1epoch')

In [9]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3) #should be 10 epochs

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.222728,3.903908,0.312278,49.595917,41:31


In [10]:
learn.save_encoder('finetuned')

In [11]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [12]:
print("\n".join(preds))

i liked this movie because it really was n't it . It was depressing and this was a wonderful movie . It was funny and about a day after the war and the end of the world . But you do n't
i liked this movie because of the fact that it was good , but the book and script were perfect . The story of Angus is difficult to follow , but as the movie progresses , he ( in his middle of a


In [13]:
dls_clas = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', text_vocab=dls_lm.vocab)

In [17]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy, model_dir='/storage/models')

In [18]:
learn = learn.load_encoder('finetuned')

In [19]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.411628,0.236804,0.904280,02:23


In [20]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.252793,0.203652,0.920640,02:47


In [21]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.199418,0.173830,0.932960,03:55


In [22]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.160032,0.182190,0.931880,05:08
1,0.140368,0.173049,0.935240,04:50
